In [1]:
import numpy as np
import os
print("Current Working Directory " , os.getcwd())

import nltk
def ExtractNP(text):
    nounphrases = []
    words = nltk.word_tokenize(text)
    tagged = nltk.pos_tag(words)

    grammar = r"""
        NP:
            {<JJ*><NN+><IN><NN>}
            {<NN.*|JJ>*<NN.*>}
        """
    chunkParser = nltk.RegexpParser(grammar)
    tree = chunkParser.parse(tagged)
    for subtree in tree.subtrees(filter=lambda t: t.label() == 'NP'):
        myPhrase = ''
        for item in subtree.leaves():
            myPhrase += ' ' + item[0]
        nounphrases.append(myPhrase.strip())
        # print(myPhrase)
    nounphrases = list(filter(lambda x: len(x.split()) > 1, nounphrases))
    return nounphrases


# from sklearn.feature_extraction.text import TfidfVectorizer
# test = TfidfVectorizer(dat1[0:10])
from collections import Counter
from tqdm import tqdm
from scipy.sparse import csr_matrix
import math
import operator
from sklearn.preprocessing import normalize
import numpy as np 

def IDF(corpus, unique_words):
    idf_dict={}
    N=len(corpus)
    for i in unique_words:
        count=0
        for sen in corpus:
            if i in sen.split():
                count=count+1
            idf_dict[i]=(math.log((1+N)/(count+1)))+1
    return idf_dict 

def fit(whole_data):
    unique_words = set()
    if isinstance(whole_data, (list,)):
        for x in whole_data:
            for y in x.split():
                if len(y)<2:
                    continue
                unique_words.add(y)
        unique_words = sorted(list(unique_words))
        vocab = {j:i for i,j in enumerate(unique_words)}
        Idf_values_of_all_unique_words=IDF(whole_data,unique_words)
    return vocab, Idf_values_of_all_unique_words

def transform(dataset,vocabulary,idf_values):
     sparse_matrix= csr_matrix((len(dataset), len(vocabulary)), dtype=np.float64)
     for row  in range(0,len(dataset)):
       number_of_words_in_sentence=Counter(dataset[row].split())
       for word in dataset[row].split():
           if word in  list(vocabulary.keys()):
               tf_idf_value=(number_of_words_in_sentence[word]/len(dataset[row].split()))*(idf_values[word])
               sparse_matrix[row,vocabulary[word]]=tf_idf_value
    #  print("NORM FORM\n",normalize(sparse_matrix, norm='l2', axis=1, copy=True, return_norm=False))
     output = normalize(sparse_matrix, norm='l2', axis=1, copy=True, return_norm=False)
     return output


import TeKET
from TeKET import Tree
from TeKET import Node
# from TeKET import Root
from TeKET import TreeManager
from TeKET import ExtractCandidate
from TeKET import NounTFCalculation

# import TeKET_comments
# from TeKET_comments import Tree
# from TeKET_comments import Node
# # from TeKET import Root
# from TeKET_comments import TreeManager
# from TeKET_comments import ExtractCandidate
# from TeKET_comments import NounTFCalculation

# import TFIDF_train_data
# from TFIDF_train_data import TFIDF
# from TFIDF_train_data import PrecisionRecallF1Calculation
# from TFIDF_train_data import DataProcessing
# import TFIDF_test_data


def find_index(word, phrase):
    i = 0
    for w in phrase:
        if w == word:
            return(i)
            break
        i = i + 1


Current Working Directory  /Users/hyoinan/OneDrive - The Ohio State University/[2019-2024 OSU]/3-2022-Spring/STAT7620/Project


In [2]:
file1 = open("task05-TRAIN/train/H-37.txt", "r")
count = 0
dat1 = np.array([])
 
while True:
    line = file1.readline()
    if not line:
        break
    dat1 = np.append(dat1, line.strip())

my_str = ""
for line in dat1:
    my_str = my_str + " " + str(line)
my_str = my_str.split(". ")
dat1 = np.array(my_str)

# candidate = np.array([])
# for sen in dat1:
#     candidate_phrase = ExtractNP(sen)
#     candidate = np.append(candidate, candidate_phrase)

# candidate = candidate.tolist()

# candidate = ExtractCandidate(my_str)
# candidate_phrases = candidate.extract_candidate_chunks()
# # print(candidate.extract_candidate_chunks())
# candidate = CleaningCandidatePhrases(candidate_phrases) # To make the phrase of the basic form

import re
from nltk.stem import PorterStemmer
ps = PorterStemmer()

def CleaningCandidatePhrases(probable_phrases):
        probpr = []
        
        for i in range (0, len(probable_phrases), 1):
            words = nltk.word_tokenize(probable_phrases[i]) 
            wa = []
            for w in words:
                x = ps.stem(w)
                wa.append(x)
            probable_phrases[i] = (' '. join(wa))
            
            #probpr.append(probable_phrases[i])
        
        for i in range (0, len(probable_phrases), 1):
            for j in range(0,len(probable_phrases[i]),1):
                regex = re.compile('[@_!#$%^&*()<>?/\|}{~:].')
                if regex.search(probable_phrases[i]) == None:
                    probpr.append(probable_phrases[i])
                    break
                else:
                    continue
                    
        for i in range (0, len(probable_phrases), 1):
            temp_probable_phrases = probable_phrases[i].split(" ")
            #print(temp_probable_phrases)
            min_length = True
            for k in range(0,len(temp_probable_phrases),1):
                if len(temp_probable_phrases[k]) < 2:
                    min_length = False
            
            
            if min_length:
                count = 0
                for j in range (0, len(probable_phrases), 1):
                    if probable_phrases[i] == probable_phrases[j]:
                        count = count + 1
            
                    if count > 2:
                        probpr.append(probable_phrases[i]) 
                        #print(probable_phrases[i])
                        break  
            else:
                continue
                    
        return probpr    

candidate = np.array([])
for sen in dat1:
    candidate_phrases = CleaningCandidatePhrases(ExtractNP(sen))
    candidate = np.append(candidate, candidate_phrases)

candidate = candidate.tolist()

Vocabulary, idf_of_vocabulary = fit(candidate) 

final_output = transform(candidate, Vocabulary, idf_of_vocabulary) # TF-IDF values

/Users/hyoinan/opt/anaconda3/envs/pytorch_env/lib/python3.8/site-packages/scipy/sparse/_index.py:82: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


In [3]:
def preparation (root, final_output, candidate):

    similar_candidate = []
    sen_ind = []
    sen_number = 0
    for sen in candidate:
        if sen.split().count(root)>0:
            similar_candidate.append(sen)
            sen_ind.append(sen_number)
        sen_number = sen_number + 1
    # # the first element will be used to construct a tree, and the rest of it will be used to grow/prune the tree

    # Solution for (1): removing dulicate words
    similar_candidate_clean = []
    for sen in similar_candidate:
        words = sen.split()
        sen = " ".join(sorted(set(words), key=words.index))
        similar_candidate_clean.append(sen)
    similar_candidate = similar_candidate_clean

    # Find TF-IDF values for the similar candidate
    index_matrix = np.array(final_output.nonzero())
    similar_TFIDF = []
    for row_ind in sen_ind:
        ind = (index_matrix[0,] == row_ind) # find the corresponding phrase
        col_ind = index_matrix[1, ind]
        tf_idf_k = np.asarray(final_output[row_ind, col_ind].todense()).flatten()
        similar_TFIDF.append(tf_idf_k)

    # Solution for (2) & (3): Removing the problematic sentences for now
    remove_list_ind = []
    for l in range(len(similar_candidate)):
        logic = (len(similar_candidate[l].split()) == len(similar_TFIDF[l]))
        if logic == False:
            # print('[phrase ' + str(l) + '] number of words: ' + str(len(similar_candidate[l].split())) + ', number of TFIDFs: ' + str(len(similar_TFIDF[l])))
            remove_list_ind.append(l)
    remove_list_ind.sort(reverse=True, key=int)
    for i in remove_list_ind:
        del similar_candidate[int(i)]
        del similar_TFIDF[int(i)]

    # print(similar_TFIDF)
    sigma1_index = similar_candidate[0].split().index(root)
    root_tfidf = similar_TFIDF[0][sigma1_index]

    return similar_candidate, similar_TFIDF, root_tfidf

In [4]:
# TF = NounTFCalculation()
# TF = TF.NounTF(candidate_phrases)
# print(TF) # possible roots?

## Get Root words

In [5]:
TF = NounTFCalculation()
TF = TF.NounTF(candidate)
# print(TF) # possible roots?

sorted_TF = sorted(TF, key=lambda tf: tf[1], reverse=True)
root_word_list = [x[0] for x in sorted_TF]
print(root_word_list)

['spam', 'svm', 'onlin', 'data', 'set', 'detect', 'perform', 'email', 'blog', 'vector', 'exampl', 'comput', 'content-bas', 'filter', 'method', 'train', 'valu', 'featur', 'high', 'cost', 'comment', 'machin', 'paramet', 'optim', 'splog', 'test', 'same', 'problem', 'text', 'relax', 'benchmark', 'time', 'learn']


In [6]:
def GetMuValueFromTree(root, mu_dict):
        if root:
            GetMuValueFromTree(root.prev, mu_dict)
            mu_dict[root.word] = root.mu
            GetMuValueFromTree(root.next, mu_dict)

def FindPhraseInCandidate(phrase, candidate_phrases):
        if len(phrase) > 0:
            for i in range (0, len(candidate_phrases)):
                if phrase == candidate_phrases[i]:
                    return i
            return -1
        else:
            return -1

In [7]:
my_final_list = []
weights = []
my_final_dict = {}

mu = -30

for i in range(len(root_word_list)):
    # print("i = " + str(i))
    root_word = root_word_list[i]
    similar_candidate, similar_TFIDF, root_tfidf = preparation(root = root_word, final_output = final_output, candidate = candidate)

    rootphrase = similar_candidate[0].split()
    rootIndex = find_index(root_word, rootphrase)

    my_tree = Tree()
    my_tree.AddNode(word = root_word, phrase = rootphrase, wordIndex = rootIndex, rootIndex = rootIndex, tfidf = root_tfidf) # Add the root first

    for j in range(len(similar_candidate)):
        phrase = similar_candidate[j].split()
        rootIndex = find_index(root_word, phrase)   # need to find rootIndex in each phrase
        # print("j = " + str(j))
        for wordIndex in range(len(phrase)):
            word = phrase[wordIndex]                    # extract the first word from the phrase with wordIndex
            word_tfidf = similar_TFIDF[j][wordIndex]    # selecting the first sentence
            
            my_tree.AddNode(word, phrase, wordIndex, rootIndex, word_tfidf) 
            # my_tree.PrintTree(my_tree.root)
            # print(str(wordIndex) + "th iteration completed...")

        my_tree.UpdateMuValues(phrase)
        
    # my_tree.PrintTree(my_tree.root)
    # my_NodeList = my_tree.CreateNodeList(my_tree.root)
    # print(my_NodeList)

    # Getting the final phrases list
    final_node_list = []
    final_node_list = my_tree.FindNodeListToExtractKeyPhrases(mu = mu, final_node_list = final_node_list, candidate_phrases= candidate) # This includes tree pruning too
    final_phrases = []
    for j in range(0, len(final_node_list)):
        final_phrases.append(my_tree.GetPhrase(final_node_list[j]))
    # my_final_list.append(final_phrases)


    ## Getting the final mu list

    # Get the mu value for each word
    mu_dict = {} # a dictionary that saves "word : mu" pairs 
    GetMuValueFromTree(my_tree.root, mu_dict)

    # Save the final keyphrases and their weights
    for l in range(0, len(final_phrases)):
        k = FindPhraseInCandidate(final_phrases[l], similar_candidate)
        if  k == -1: 
            continue
        else:
            my_final_list.append(final_phrases[l])

            # calculate the sum of tfidf of all the words in the final keyphrase
            tfidf_value = np.sum(similar_TFIDF[k])
            # calculate the sum of mu of all the words in the final keyphrase
            mu_value = 0 
            for j in final_phrases[l].split():
                mu_value = mu_value + mu_dict[j]
            weights.append(tfidf_value * mu_value)
    
    for m in range(0, len(my_final_list)): 
        my_final_dict[my_final_list[m]] = weights[m]
    print(str(i+1) + "th iteration completed...")

1th iteration completed...
2th iteration completed...
3th iteration completed...
4th iteration completed...
5th iteration completed...
6th iteration completed...
7th iteration completed...
8th iteration completed...
9th iteration completed...
10th iteration completed...
11th iteration completed...
12th iteration completed...
13th iteration completed...
14th iteration completed...
15th iteration completed...
16th iteration completed...
17th iteration completed...
18th iteration completed...
19th iteration completed...
20th iteration completed...
21th iteration completed...
22th iteration completed...
23th iteration completed...
24th iteration completed...
25th iteration completed...
26th iteration completed...
27th iteration completed...
28th iteration completed...
29th iteration completed...
30th iteration completed...
31th iteration completed...
32th iteration completed...
33th iteration completed...


In [8]:
my_final_dict_sorted = dict(sorted(my_final_dict.items(), key=lambda item: item[1], reverse = True))
my_final_dict_sorted

{'contentbas spam': 98.63966598337315,
 'content-bas spam detect': 89.02817394173105,
 'large-scal content-bas spam detect': 76.08708019031403,
 'spam detect': 70.14648337825203,
 'data mine': 68.25308995798811,
 'svm': 65.0,
 'spam comment': 42.958887237994766,
 'blog comment spam detect': 39.57373025243774,
 'compar perform': 37.412963192279804,
 'activ set': 36.592029755266175,
 'spam filter': 36.10039606681223,
 'blog identif': 34.500322092369515,
 'benchmark data set': 29.085154391608146,
 'comment spam detect experi': 27.14284343758393,
 'expens comput': 26.66738746367917,
 'uniqu spam filter': 23.381332591668198,
 'high level': 22.224111389664692,
 'onlin spam filter': 20.49895621650174,
 'optim stabil': 19.525668558330533,
 'method trade': 19.50971486029461,
 'suport vector machin': 18.6785376848127,
 'vector machin': 18.364134580450788,
 'machin learn': 16.963742278093555,
 'classifi exampl': 16.75538770736061,
 'costli method': 15.249556668721373,
 'comput save': 15.235192819

In [9]:
# key = list(my_final_dict_sorted.keys())
# value = list(my_final_dict_sorted.values())

# final_real_phrases = {}
# for i in range(len(key)):
#     if len(key[i].split()) > 1:
#         final_real_phrases[key[i]] = value[i]
# final_real_phrases

In [10]:
final_real_phrases = my_final_dict_sorted

N = 15
N_phrases = list(final_real_phrases.keys())[0:N]
print("(2) Using ExtractNP()")
N_phrases

(2) Using ExtractNP()


['contentbas spam',
 'content-bas spam detect',
 'large-scal content-bas spam detect',
 'spam detect',
 'data mine',
 'svm',
 'spam comment',
 'blog comment spam detect',
 'compar perform',
 'activ set',
 'spam filter',
 'blog identif',
 'benchmark data set',
 'comment spam detect experi',
 'expens comput']

In [11]:
# H-37: K_correct = 1
standard = ['support vector machin', 'content-base filter', 'spam filter', 'blog', 'splog', 'link analysi', 
'machin learn techniqu', 'link spam', 'content-base spam detect', 'bayesian method', 'increment updat', 
'logist regress', 'hyperplan', 'featur map', 'spam filter']
N_standard = len(standard)
standard

['support vector machin',
 'content-base filter',
 'spam filter',
 'blog',
 'splog',
 'link analysi',
 'machin learn techniqu',
 'link spam',
 'content-base spam detect',
 'bayesian method',
 'increment updat',
 'logist regress',
 'hyperplan',
 'featur map',
 'spam filter']

In [12]:
# # H-44: K_correct = 2
# standard = "time machin,text search,invert file index,tempor search,approxim tempor coalesc,web archiv,version document collect,collabor author environ,timestamp inform feed,document-content overlap,index rang-base valu,open sourc search-engin nutch,static indexprun techniqu,valid time-interv,sublist materi,tempor text index,time-travel text search"
# standard = standard.split(',')
# N_standard = len(standard)
# standard

In [13]:
# # H-49: K_correct = 3 (N = 15)
# standard = "perform predict,inform retriev,spatial autocorrel,autocorrel,cluster hypothesi,zero relev judgment,relationship of predictor+predictor relationship,predict power of predictor+predictor predict power,languag model score,rank of queri+queri rank,regular"
# standard = standard.split(',')
# N_standard = len(standard)
# standard

In [14]:
# # H-98: K_correct = 2
# standard = "bayesian risk model,empir score distribut,probabl estim,text classifi,parametr model,inform retriev,logist regress framework,posterior function,asymmetr laplac distribut,search engin retriev,symmetr distribut,asymmetr gaussian,maximum likelihood estim,class-condit densiti,decis threshold,text classif,cost-sensit learn,activ learn,classifi combin"
# standard = standard.split(',')

In [15]:
# # J=61: K_correct = 1
# standard = "iter combinatori exchang,doubl auction,combinatori auction,buyer and seller,trade,tree-base bid languag,price,winner-determin,bid,threshold payment,combinatori exchang,vcg,prefer elicit"
# standard = standard.split(',')

In [16]:
K_correct = 0
correct_words = []
for word in N_phrases:
    if word in standard:
        K_correct = K_correct + 1
        correct_words.append(word)
print('Precision: ' + str(K_correct/N))
print('Recall: ' + str(K_correct/N_standard))
print('F1 Score: ' + str((2 * K_correct/N * K_correct/N_standard)/(K_correct/N + K_correct/N_standard)))
correct_words

Precision: 0.06666666666666667
Recall: 0.06666666666666667
F1 Score: 0.06666666666666667


['spam filter']